# Import Dependencies

In [1]:
import numpy as np
import pandas as pd

pd.set_option('max_columns', 35)

# Import Data

In [2]:
customers_df = pd.read_csv("../data/raw/customers.csv")
vendors_df = pd.read_csv("../data/raw/vendors.csv")
locations_df = pd.read_csv("../data/raw/locations.csv")
orders_df = pd.read_csv("../data/raw/orders.csv")

# Data Cleaning

## Drop Duplicate Customer Observations

In [3]:
customers_df = customers_df.drop_duplicates(subset=['akeed_customer_id'], keep='first')

## Drop Vendor Columns Specified in the Assessment Brief

In [4]:
vendors_df = vendors_df.drop([
    'sunday_from_time1', 'sunday_to_time1', 'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1',
    'monday_to_time1', 'monday_from_time2', 'monday_to_time2', 'tuesday_from_time1', 'tuesday_to_time1',
    'tuesday_from_time2', 'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1', 'wednesday_from_time2',
    'wednesday_to_time2', 'thursday_from_time1', 'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
    'friday_from_time1', 'friday_to_time1', 'friday_from_time2', 'friday_to_time2', 'saturday_from_time1',
    'saturday_to_time1', 'saturday_from_time2', 'saturday_to_time2'
], axis=1)

## Drop Orders with Empty `customer_id` or `item_count` Fields

In [5]:
orders_df['customer_id'].replace('', np.nan, inplace=True)
orders_df['item_count'].replace('', np.nan, inplace=True)
orders_df = orders_df.dropna(subset=['customer_id', 'item_count'])

# Refactoring the Indexing System
## Rename Legacy `ID` Columns
We'll be creating new ID columns which use incremental integers. This new system will be the same across all tables; the current system uses a number of different indexing methods.

In [6]:
customers_df = customers_df.rename(columns={'akeed_customer_id': 'legacy_customer_id'})
vendors_df = vendors_df.rename(columns={'id': 'legacy_vendor_id'})
locations_df = locations_df.rename(columns={'customer_id': 'legacy_customer_id'})
orders_df = orders_df.rename(columns={
    'akeed_order_id': 'legacy_order_id', 'customer_id': 'legacy_customer_id', 'vendor_id': 'legacy_vendor_id'
})

## Create a New `customer_id`

In [7]:
customers_df['customer_id'] = customers_df.index

### Copy the new `customer_id` to the other dataframes

In [8]:
# create a dict of 'legacy_customer_id' and the corresponding 'customer_id'
customer_ids = customers_df.set_index('legacy_customer_id')['customer_id'].to_dict()

# copy the new 'customer_id' values to 'locations_df' and 'orders_df'
locations_df['customer_id'] = locations_df['legacy_customer_id'].map(customer_ids)
orders_df['customer_id'] = orders_df['legacy_customer_id'].map(customer_ids)

# drop 'legacy_customer_id' from 'customers_df', 'locations_df', and 'orders_df'
customers_df = customers_df.drop(columns=['legacy_customer_id'], axis=1)
locations_df = locations_df.drop(columns=['legacy_customer_id'], axis=1)
orders_df = orders_df.drop(columns=['legacy_customer_id'], axis=1)

We can now see that there are a number of missing `customer_id` values in `locations_df` and `orders_df`; this is because certain locations/orders pertain to customers which once existed in the dataset, but no longer do:

In [9]:
print("Missing 'customer_id' count in 'locations_df': ", locations_df['customer_id'].isnull().sum())
print("Missing 'customer_id' count in 'orders_df': ", orders_df['customer_id'].isnull().sum())

Missing 'customer_id' count in 'locations_df':  1479
Missing 'customer_id' count in 'orders_df':  3101


...we can drop these records:

In [10]:
locations_df = locations_df.dropna(subset=['customer_id'])
orders_df = orders_df.dropna(subset=['customer_id'])

## Create a New `vendor_id`

In [11]:
vendors_df['vendor_id'] = vendors_df.index

### Copy the new `vendor_id` to the other dataframes

In [12]:
# create a dict of 'legacy_vendor_id' and the corresponding 'vendor_id'
vendor_ids = vendors_df.set_index('legacy_vendor_id')['vendor_id'].to_dict()

# copy the new 'vendor_id' to 'orders_df'
orders_df['vendor_id'] = orders_df['legacy_vendor_id'].map(vendor_ids)

# drop 'legacy_vendor_id' from 'vendors_df' and 'orders_df'
vendors_df = vendors_df.drop(columns=['legacy_vendor_id'], axis=1)
orders_df = orders_df.drop(columns=['legacy_vendor_id'], axis=1)

## Create a New `order_id`

In [13]:
orders_df['order_id'] = orders_df.index

# drop 'legacy_order_id' from 'orders_df'
orders_df = orders_df.drop(columns=['legacy_order_id'], axis=1)

## Create a New `location_id`

In [14]:
locations_df['location_id'] = locations_df.index

### Copy the new `location_id` to the other dataframes

In [15]:
def update_location_id(customer_id, location_number):
    return locations_df.loc[
        (locations_df['customer_id'] == customer_id) & (locations_df['location_number'] == location_number)
    ]['location_id'].values[0]

orders_df['location_id'] = [update_location_id(*a) for a in tuple(
    zip(orders_df['customer_id'], orders_df['LOCATION_NUMBER'])
)]

In [16]:
# drop 'location_number' from 'locations_df'
locations_df = locations_df.drop(columns=['location_number'])

# drop 'LOCATION_NUMBER' and 'LOCATION_TYPE' from 'orders_df'
orders_df = orders_df.drop(columns=['LOCATION_NUMBER', 'LOCATION_TYPE'])

# More Data Cleaning
Now that we've refactored the ID system, we can continue with the data cleaning.

## Customer Data
There doesn't seem to be much to do here. It's worth noting that there is a significant amount of missing values in `gender`, `dob`, and `language`. In order to fix this, the company would have to reach out to the customers in question and ask them to update their details.

In [17]:
customers_df.isnull().sum()

gender         12154
dob            31477
status             0
verified           0
language       13424
created_at         0
updated_at         0
customer_id        0
dtype: int64

We can rename `dob`, as it holds customers' year of birth rather than a specific 'date of birth':

In [18]:
customers_df = customers_df.rename(columns={'dob': 'birth_year'})

## Vendor Data
`authentication_id` is a float, however it contains integer-like values; let's fix this:

_There are a number of columns which also share this issue, however they will be removed later on so it would be pointless changing them here._

In [19]:
vendors_df['authentication_id'] = vendors_df['authentication_id'].astype(int)

We can drop the `primary_tags` column as it doesn't seem to correspond with any other data we have here. It also invalidates the vendor table's second normal form.

_`vendor_tag` and `vendor_tag_name` also invalidates the vendor table's 2NF, however we'll be extracting these values into their own tables shortly._

In [20]:
vendors_df = vendors_df.drop(columns=['primary_tags'], axis=1)

The `is_open` column can be dropped as it seems redundant - whether a vendor is open or not can be determined on the front end by checking the opening time columns. The same reasoning can be applied to the removal of `open_close_flags`.

The `country_id` and `city_id` columns seem to reference tables which aren't present in our data, so they can be dropped as well.

In [21]:
vendors_df = vendors_df.drop(columns=['is_open', 'open_close_flags', 'country_id', 'city_id'], axis=1)

Finally, let's rename some of the columns for the sake of readability:

In [22]:
vendors_df = vendors_df.rename(columns={
    'authentication_id': 'auth_id', 'delivery_charge': 'delivery_fee', 'serving_distance': 'max_serving_dist',
    'OpeningTime': 'opening_time', 'OpeningTime2': 'opening_time_2', 'prepration_time': 'preparation_time',
    'vendor_rating': 'avg_vendor_rating', 'one_click_vendor': 'is_one_click'
})

Before we continue, it's worth noting that the `commission`, `language`, `vendor_tag`, and `vendor_tag_name` columns contain missing values. These would need to be corrected in collaboration with the vendors in question.

## Location Data
There isn't much to do here. `customer_id` is of type float, when it should be int - let's fix this:

In [23]:
locations_df['customer_id'] = locations_df['customer_id'].astype(int)

## Order Data
Let's start by correcting the `item_count` dtype:

In [24]:
orders_df['item_count'] = orders_df['item_count'].astype(int)

Next, the `delivery_time` column doesn't appear to be providing any value as very few columns have a valid entry in this field, so we can drop it:

_The same can be said for `promo_code`, however we'll keep this column and move it to it's own table for the sake of example._

In [25]:
print("Total number of records in 'orders' table: ", orders_df.shape[0])
print("Missing values in 'delivery_time': ", orders_df['delivery_time'].isnull().sum())

orders_df = orders_df.drop(columns=['delivery_time'], axis=1)

Total number of records in 'orders' table:  125277
Missing values in 'delivery_time':  120569


`driver_rating` pertains to the delivery driver assigned to the order. As we don't have a table listing driver information, we can drop this column:

In [26]:
orders_df = orders_df.drop(columns=['driver_rating'], axis=1)

The `CID X LOC_NUM X VENDOR` column is useless in the context of this assignment as it's used as a submission for the Kaggle challenge from which this dataset originates. Let's drop it:

In [27]:
orders_df = orders_df.drop(columns=['CID X LOC_NUM X VENDOR'], axis=1)

Finally, let's fix the `customer_id` dtype:

In [28]:
orders_df['customer_id'] = orders_df['customer_id'].astype(int)

# Extracting Vendor Category Information
Despite all of the vendors in our dataset having a single category, we can extract `vendor_category_en` into its own table. This would allow for a vendor to have multiple categories in the future which would enhance the user search experience. (_Some vendors might be a restaurant which also sells sweets & bakes, for example_)

We must also create a second table to house the relationships between vendors and their categories.

### Create a table which contains each category and an identifier

In [29]:
vendor_categories_df = vendors_df[['vendor_category_en']].copy().rename(
    columns={'vendor_category_en': 'category'}
).drop_duplicates(keep='first').reset_index(drop=True)

# use the pandas index as 'category_id'
vendor_categories_df['category_id'] = vendor_categories_df.index

### Create a table which contains the relationships between vendors and their categories

In [30]:
# copy 'vendor_id' and 'vendor_category_en' into a new dataframe
vendor_category_relationships_df = vendors_df[['vendor_id', 'vendor_category_en']].copy().rename(
    columns={'vendor_category_en': 'category'}
)

# create a dict of 'category' and 'category_id' from 'vendor_categories_df'
category_ids = vendor_categories_df.set_index('category')['category_id'].to_dict()

# replace the categories with their respective ID
vendor_category_relationships_df = vendor_category_relationships_df.replace({'category': category_ids}).rename(
    columns={'category': 'category_id'}
)

# drop the category columns from 'vendors_df'
vendors_df = vendors_df.drop(columns=['vendor_category_en', 'vendor_category_id'], axis=1)

# Extracting Vendor Tags Information

### Create a table which contains each tag and an identifier

In [31]:
# copy 'vendor_tag_name from 'vendors_df'
vendor_tags_df = vendors_df[['vendor_tag_name']].copy().rename(
    columns={'vendor_tag_name': 'tag'}
)

# split tags into lists and explode those lists into new rows for each item
vendor_tags_df['tag'] = vendor_tags_df['tag'].str.split(',').tolist()
vendor_tags_df = vendor_tags_df.explode('tag').drop_duplicates(keep='first').reset_index(drop=True)

# use the pandas index as 'tag_id'
vendor_tags_df['tag_id'] = vendor_tags_df.index

### Create a table which contains the relationships between vendors and their categories

In [32]:
# copy 'vendor_id' and 'vendor_tag_name' from 'vendors_df'
vendor_tag_relationships_df = vendors_df[['vendor_id', 'vendor_tag_name']].copy().rename(
    columns={'vendor_tag_name': 'tag'}
)

# explode the lists of tags as before
vendor_tag_relationships_df['tag'] = vendor_tag_relationships_df['tag'].str.split(',').tolist()
vendor_tag_relationships_df = vendor_tag_relationships_df.explode('tag').reset_index(drop=True)

# create a dict of 'tag' and 'tag_id' from 'vendor_tags_df'
tag_ids = vendor_tags_df.set_index('tag')['tag_id'].to_dict()

# replace the tags with their respective ID
vendor_tag_relationships_df = vendor_tag_relationships_df.replace({'tag': tag_ids}).rename(
    columns={'tag': 'tag_id'}
)

# drop the tag columns from 'vendors_df'
vendors_df = vendors_df.drop(columns=['vendor_tag', 'vendor_tag_name'], axis=1)

# Extract Vendor Ratings Information

In [33]:
# copy 'vendor_id', 'customer_id', and 'vendor_rating' from 'orders_df'
vendor_ratings_df = orders_df[['vendor_id', 'customer_id', 'vendor_rating']].copy().rename(
    columns={'vendor_rating': 'rating'}
)

# remove records with missing 'rating' fields, or those where 'rating' == 0
vendor_ratings_df['rating'].replace(0.0, np.nan, inplace=True)
vendor_ratings_df = vendor_ratings_df.dropna(subset=['rating']).reset_index(drop=True)

# drop 'vendor_rating' from 'orders_df'
orders_df = orders_df.drop(['vendor_rating'], axis=1)

# Extract Promo Codes Information

In [34]:
# copy 'promo_code' and 'promo_code_discount_percentage' from 'orders_df'
promo_codes_df = orders_df[['promo_code', 'promo_code_discount_percentage']].copy().rename(
    columns={'promo_code': 'code', 'promo_code_discount_percentage': 'discount_percentage'}
)

# convert all codes to lowercase
promo_codes_df['code'] = promo_codes_df['code'].str.lower()

# remove missing codes and those with missing 'discount_percentage'
promo_codes_df['code'].replace('', np.nan, inplace=True)
promo_codes_df['discount_percentage'].replace(0.0, np.nan, inplace=True)
promo_codes_df = promo_codes_df.dropna(subset=['code', 'discount_percentage'])

# remove duplicate codes
promo_codes_df = promo_codes_df.drop_duplicates(subset=['code'], keep='first')

# use the pandas index as 'code_id'
promo_codes_df['code_id'] = promo_codes_df.index

# create a dict of 'code' and corresponding 'code_id'
code_ids = promo_codes_df.set_index('code')['code_id'].to_dict()

# copy new 'code_id' into 'orders_df'
orders_df['promo_code'] = orders_df['promo_code'].str.lower()
orders_df['promo_code_id'] = orders_df['promo_code'].map(code_ids)

# drop 'promo_code' and 'promo_code_discount_percentage' from 'orders_df'
orders_df = orders_df.drop(['promo_code', 'promo_code_discount_percentage'], axis=1)

# Extract Customer Favorite Vendors Information

In [38]:
# copy 'customer_id' and 'vendor_id' from 'orders_df', where 'is_favorite' == 'Yes'
customer_fav_vendors_df = orders_df[orders_df['is_favorite'] == 'Yes'][
    ['customer_id', 'vendor_id']
].copy().reset_index(drop=True)

# drop 'is_favorite' from 'orders_df'
orders_df = orders_df.drop(['is_favorite'], axis=1)

# Finishing Touches
## Renaming Order Columns

In [42]:
orders_df = orders_df.rename(columns={
    'item_count': 'n_items', 'grand_total': 'price_due', 'payment_mode': 'payment_method',
    'deliverydistance': 'delivery_distance', 'preparationtime': 'preparation_time'
})

## Exporting

In [44]:
customers_df.to_csv("../data/customers.csv")
vendors_df.to_csv("../data/vendors.csv")
locations_df.to_csv("../data/locations.csv")
orders_df.to_csv("../data/orders.csv")

vendor_categories_df.to_csv("../data/vendor_categories.csv")
vendor_category_relationships_df.to_csv("../data/vendor_category_relationships.csv")
vendor_tags_df.to_csv("../data/vendor_tags.csv")
vendor_tag_relationships_df.to_csv("../data/vendor_tag_relationships.csv")
vendor_ratings_df.to_csv("../data/vendor_ratings.csv")
promo_codes_df.to_csv("../data/promo_codes.csv")
customer_fav_vendors_df.to_csv("../data/customer_fav_vendors.csv")